# Contact
For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

- Federico Ruggeri -> federico.ruggeri6@unibo.it
- Eleonora Mancini -> e.mancini@unibo.it

Professor:
- Paolo Torroni -> p.torroni@unibo.it

# Introduction
You are asked to address the [EXIST 2023 Task 1](https://clef2023.clef-initiative.eu/index.php?page=Pages/labs.html#EXIST) on sexism detection.

## Problem Definition
The systems have to decide whether or not a given tweet contains or describes sexist expressions or behaviors (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behavior).

### Examples:

**Text**: *Can’t go a day without women womening*

**Label**: Sexist

**Text**: *''Society's set norms! Happy men's day though!#weareequal''*

**Label**: Not sexist

#[Task 1 - 1.0 points] Corpus

We have preparared a small version of EXIST dataset in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material/tree/main/2024-2025/Assignment%201/data).

Check the `A1/data` folder. It contains 3 `.json` files representing `training`, `validation` and `test` sets.

The three sets are slightly unbalanced, with a bias toward the `Non-sexist` class.



### Dataset Description
- The dataset contains tweets in both English and Spanish.
- There are labels for multiple tasks, but we are focusing on **Task 1**.
- For Task 1, soft labels are assigned by six annotators.
- The labels for Task 1 represent whether the tweet is sexist ("YES") or not ("NO").







### Example


    "203260": {
        "id_EXIST": "203260",
        "lang": "en",
        "tweet": "ik when mandy says “you look like a whore” i look cute as FUCK",
        "number_annotators": 6,
        "annotators": ["Annotator_473", "Annotator_474", "Annotator_475", "Annotator_476", "Annotator_477", "Annotator_27"],
        "gender_annotators": ["F", "F", "M", "M", "M", "F"],
        "age_annotators": ["18-22", "23-45", "18-22", "23-45", "46+", "46+"],
        "labels_task1": ["YES", "YES", "YES", "NO", "YES", "YES"],
        "labels_task2": ["DIRECT", "DIRECT", "REPORTED", "-", "JUDGEMENTAL", "REPORTED"],
        "labels_task3": [
          ["STEREOTYPING-DOMINANCE"],
          ["OBJECTIFICATION"],
          ["SEXUAL-VIOLENCE"],
          ["-"],
          ["STEREOTYPING-DOMINANCE", "OBJECTIFICATION"],
          ["OBJECTIFICATION"]
        ],
        "split": "TRAIN_EN"
      }
    }

### Instructions
1. **Download** the `A1/data` folder.
2. **Load** the three JSON files and encode them as pandas dataframes.
3. **Generate hard labels** for Task 1 using majority voting and store them in a new dataframe column called `hard_label_task1`. Items without a clear majority will be removed from the dataset.
4. **Filter the DataFrame** to keep only rows where the `lang` column is `'en'`.
5. **Remove unwanted columns**: Keep only `id_EXIST`, `lang`, `tweet`, and `hard_label_task1`.
6. **Encode the `hard_label_task1` column**: Use 1 to represent "YES" and 0 to represent "NO".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import numpy as np
import nltk

from tqdm import tqdm
from collections import OrderedDict
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import WhitespaceTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Reading JSON data from a file
def encode_json(file_path):
  with open(file_path, 'r', encoding='utf-8') as file:
      data = json.load(file)

  df = pd.DataFrame.from_dict(data, orient='index')
  return df

# Convert dictionary to pandas DataFrame
training = encode_json('/content/drive/MyDrive/assignment-2425/data/training.json')
validation = encode_json('/content/drive/MyDrive/assignment-2425/data/validation.json')
test = encode_json('/content/drive/MyDrive/assignment-2425/data/test.json')

In [ ]:
def get_labels(soft_labels):
  labels = []
  for row in soft_labels['labels_task1']:
    if row.count('YES') > row.count('NO'):
      labels.append('YES')
    elif row.count('NO') > row.count('YES'):
      labels.append('NO')
    else:
      labels.append(None)

  return labels

training['hard_label_task1'] = get_labels(training)
validation['hard_label_task1'] = get_labels(validation)
test['hard_label_task1'] = get_labels(test)

training.dropna(inplace=True)
validation.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
training = training[training['lang'] == 'en']
validation = validation[validation['lang'] == 'en']
test = test[test['lang'] == 'en']

In [ ]:
training = training.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
validation = validation.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]
test = test.loc[:,['id_EXIST', 'lang', 'tweet', 'hard_label_task1']]

In [ ]:
training.loc[training['hard_label_task1'] == 'YES','hard_label_task1'] = 1
training.loc[training['hard_label_task1'] == 'NO','hard_label_task1'] = 0

validation.loc[validation['hard_label_task1'] == 'YES','hard_label_task1'] = 1
validation.loc[validation['hard_label_task1'] == 'NO','hard_label_task1'] = 0

test.loc[test['hard_label_task1'] == 'YES','hard_label_task1'] = 1
test.loc[test['hard_label_task1'] == 'NO','hard_label_task1'] = 0

# [Task2 - 0.5 points] Data Cleaning
In the context of tweets, we have noisy and informal data that often includes unnecessary elements like emojis, hashtags, mentions, and URLs. These elements may interfere with the text analysis.



### Instructions
- **Remove emojis** from the tweets.
- **Remove hashtags** (e.g., `#example`).
- **Remove mentions** such as `@user`.
- **Remove URLs** from the tweets.
- **Remove special characters and symbols**.
- **Remove specific quote characters** (e.g., curly quotes).
- **Perform lemmatization** to reduce words to their base form.

In [ ]:
import re

def clean_tweet(tweet):
    # Remove emojis
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)

    # Remove hashtags
    tweet = re.sub(r'#\w+', '', tweet)

    # Remove mentions
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    # Remove special characters and symbols
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove specific quote characters
    tweet = re.sub(r'[“”‘’"\']', '', tweet)

    # Remove excessive whitespaces
    tweet = re.sub(r'\s+', ' ', tweet)

    return tweet.strip().lower()

training['tweet'] = training['tweet'].apply(clean_tweet)
validation['tweet'] = validation['tweet'].apply(clean_tweet)
test['tweet'] = test['tweet'].apply(clean_tweet)

train_texts = training['tweet'].values
val_texts = validation['tweet'].values
test_texts = test['tweet'].values

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

def get_wordnet_key(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

def lem_text(text: str):
    tokens = tokenizer.tokenize(text)
    tagged = pos_tag(tokens)
    words = [lemmatizer.lemmatize(word, get_wordnet_key(tag)) for word, tag in tagged]
    return " ".join(words)

lem_train_texts = [lem_text(text) for text in tqdm(train_texts, leave=True, position=0)]
lem_val_texts = [lem_text(text) for text in tqdm(val_texts, leave=True, position=0)]
lem_test_texts = [lem_text(text) for text in tqdm(test_texts, leave=True, position=0)]

training['lemmatized'] = lem_train_texts
validation['lemmatized'] = lem_val_texts
test['lemmatized'] = lem_test_texts

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
100%|██████████| 286/286 [00:00<00:00, 886.28it/s]


# [Task 3 - 0.5 points] Text Encoding
To train a neural sexism classifier, you first need to encode text into numerical format.




### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.





### Note : What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe **must** be added to the vocabulary.
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **special token** (e.g., [UNK]) and a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, etc...)



### More about OOV

For a given token:

* **If in train set**: add to vocabulary and assign an embedding (use GloVe if token in GloVe, custom embedding otherwise).
* **If in val/test set**: assign special token if not in vocabulary and assign custom embedding.

Your vocabulary **should**:

* Contain all tokens in train set; or
* Union of tokens in train set and in GloVe $\rightarrow$ we make use of existing knowledge!

In [ ]:
import gensim
import gensim.downloader as gloader

embedding_dimension = 100
glove = gloader.load("glove-wiki-gigaword-{}".format(embedding_dimension))

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
def build_vocabulary(df):
    idx_to_word = OrderedDict()
    word_to_idx = OrderedDict()

    # start from 2 because index 0 will be for padding, 1 for <OOV>
    curr_idx = 2
    for sentence in df:
        tokens = sentence.split()
        for token in tokens:
            if token not in word_to_idx:
                word_to_idx[token] = curr_idx
                idx_to_word[curr_idx] = token
                curr_idx += 1

    word_listing = list(idx_to_word.values())
    return idx_to_word, word_to_idx, word_listing

idx_to_word, word_to_idx, word_listing = build_vocabulary(lem_train_texts)
#add 'UNK' token for the OOV tokens in val and test sets
idx_to_word[1] = 'UNK'
word_to_idx['UNK'] = 1
word_listing.append('UNK')
print(f"\n{len(word_listing)}")


9841


In [ ]:
#tokenizing the tweets
def tokenize_and_map(text, word_to_index):

    tokens = text.split()

    # 4. Map tokens to indices, use 'UNK' index if not found
    indices = [word_to_index.get(token, word_to_index['UNK']) for token in tokens]

    return indices

training['tokenized'] = training['lemmatized'].apply(lambda x: tokenize_and_map(x, word_to_idx))
validation['tokenized'] = validation['lemmatized'].apply(lambda x: tokenize_and_map(x, word_to_idx))
test['tokenized'] = test['lemmatized'].apply(lambda x: tokenize_and_map(x, word_to_idx))

In [ ]:
def check_OOV_terms(embedding_model,
                    word_listing):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """
    embedding_vocabulary = set(embedding_model.key_to_index.keys())
    oov = set(word_listing).difference(embedding_vocabulary)
    return list(oov)

oov_terms = check_OOV_terms(glove, word_listing)
oov_percentage = float(len(oov_terms)) * 100 / len(word_listing)
print(f"Total OOV terms: {len(oov_terms)} ({oov_percentage:.2f}%)")

Total OOV terms: 1865 (18.95%)


random embeddings

In [ ]:
def build_embedding_matrix(embedding_model,
                           embedding_dimension,
                           word_to_idx,
                           vocab_size,
                           oov_terms):
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param vocab_size: size of the vocabulary
    :param oov_terms: list of OOV terms (list)

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dimension), dtype=np.float32)
    for word, idx in tqdm(word_to_idx.items()):
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [ ]:
embedding_matrix = build_embedding_matrix(glove, embedding_dimension, word_to_idx, len(word_to_idx)+1, oov_terms)
np.save('/content/drive/MyDrive/assignment-2425/embed.npy', embedding_matrix)
print(f"\nEmbedding matrix shape: {embedding_matrix.shape}")

100%|██████████| 9841/9841 [00:00<00:00, 292055.09it/s]


Embedding matrix shape: (9842, 100)


# [Task 4 - 1.0 points] Model definition

You are now tasked to define your sexism classifier.




### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.

### Token to embedding mapping

You can follow two approaches for encoding tokens in your classifier.

### Work directly with embeddings

- Compute the embedding of each input token
- Feed the mini-batches of shape (batch_size, # tokens, embedding_dim) to your model

### Work with Embedding layer

- Encode input tokens to token ids
- Define a Embedding layer as the first layer of your model
- Compute the embedding matrix of all known tokens (i.e., tokens in your vocabulary)
- Initialize the Embedding layer with the computed embedding matrix
- You are **free** to set the Embedding layer trainable or not

In [ ]:
#Need to pad tokenized sequences to max_length, 60 is the max_length in training
max_len = 60
training['padded'] = pad_sequences(training['tokenized'], maxlen=max_len, padding='post').tolist()
validation['padded'] = pad_sequences(validation['tokenized'], maxlen=max_len, padding='post').tolist()
test['padded'] = pad_sequences(test['tokenized'], maxlen=max_len, padding='post').tolist()

In [ ]:
embedding_matrix = np.load('/content/drive/MyDrive/assignment-2425/embed.npy')
vocab_size = embedding_matrix.shape[0]

baseline = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dimension,
        weights=[embedding_matrix],  # Load the pre-trained embeddings
        mask_zero=True,
        trainable=False               # Set to False to keep embeddings fixed
    ),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(1, activation='sigmoid')  # For binary classification
])

model_1 = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dimension,
        weights=[embedding_matrix],  # Load the pre-trained embeddings
        mask_zero=True,
        trainable=False               # Set to False to keep embeddings fixed
    ),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)), # add another LSTM layer
    Dense(1, activation='sigmoid')  # For binary classification
])

### Padding

Pay attention to padding tokens!

Your model **should not** be penalized on those tokens.

#### How to?

There are two main ways.

However, their implementation depends on the neural library you are using.

- Embedding layer
- Custom loss to compute average cross-entropy on non-padding tokens only

**Note**: This is a **recommendation**, but we **do not penalize** for missing workarounds.

# [Task 5 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate the Baseline and Model 1.



### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.
* Evaluate your models using macro F1-score.

In [ ]:
x_train = np.array(training['padded'].to_list()).astype('float32')
y_train = training['hard_label_task1'].values.astype('float32')

x_val = np.array(validation['padded'].to_list()).astype('float32')
y_val = validation['hard_label_task1'].values.astype('float32')

In [ ]:
np.expand_dims(y_val,1).shape

(158, 1)

In [ ]:
seed_values = [1, 42 , 7]

for seed in seed_values:
  print(f"model_{seed}")
  tf.random.set_seed(seed)

  optim = tf.keras.optimizers.AdamW(learning_rate=0.001)
  reduce_LR = tf.keras.callbacks.ReduceLROnPlateau(patience=4)
  checkpoint_baseline = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/assignment-2425/baseline_trainable_{seed}.weights.h5', save_weights_only=True, save_best_only=True)

  baseline = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dimension,
        weights=[embedding_matrix],  # Load the pre-trained embeddings
        mask_zero=True,
        trainable=True               # Set to False to keep embeddings fixed
    ),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(1, activation='sigmoid')  # For binary classification
])

  baseline.compile(
      loss='binary_crossentropy',
      optimizer=optim,
      metrics=['accuracy']
  )

  history_baseline = baseline.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[reduce_LR, checkpoint_baseline]
  )

  baseline.load_weights(f'/content/drive/MyDrive/assignment-2425/baseline_trainable_{seed}.weights.h5')
  y_pred = baseline.predict(x_val)
  f1 = tf.keras.metrics.F1Score(threshold=0.5, average='macro')
  f1.update_state(np.expand_dims(y_val,1).astype('int32'), y_pred)
  print(f1.result())

model_1
Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6160 - loss: 0.6470 - val_accuracy: 0.6456 - val_loss: 0.7049 - learning_rate: 0.0010
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7367 - loss: 0.5331 - val_accuracy: 0.7342 - val_loss: 0.7174 - learning_rate: 0.0010
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8140 - loss: 0.4255 - val_accuracy: 0.7658 - val_loss: 0.5174 - learning_rate: 0.0010
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8698 - loss: 0.3092 - val_accuracy: 0.7722 - val_loss: 0.5040 - learning_rate: 0.0010
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9160 - loss: 0.2141 - val_accuracy: 0.8038 - val_loss: 0.5691 - learning_rate: 0.0010
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9557 - loss: 0.1209 - val_accuracy: 0.7722 - val_loss: 0.5630 - learning_rate: 0.0010
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9763 - loss: 0.0690 -

#### Embedding(training=False) F1-scores = (0.7633587, 0.76422757, 0.779661)
#### Embedding(trainable=True) F1-score = (0.7428571, 0.72580636, 0.73015875)

In [ ]:
seed_values = [1, 42 , 7]

for seed in seed_values:
  print(f"model_{seed}")
  tf.random.set_seed(seed)

  model_1 = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dimension,
        weights=[embedding_matrix],  # Load the pre-trained embeddings
        mask_zero=True,
        trainable=True               # Set to False to keep embeddings fixed
    ),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)), # add another LSTM layer
    Dense(1, activation='sigmoid')  # For binary classification
  ])

  optim = tf.keras.optimizers.AdamW(learning_rate=0.001)
  reduce_LR = tf.keras.callbacks.ReduceLROnPlateau(patience=4)
  checkpoint_model1 = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/assignment-2425/model1_trainable_{seed}.weights.h5', save_weights_only=True, save_best_only=True)

  model_1.compile(
      loss='binary_crossentropy',
      optimizer=optim,
      metrics=['accuracy']
  )

  history_model1 = model_1.fit(
      x_train,
      y_train,
      epochs=30,
      batch_size=32,
      validation_data=(x_val, y_val),
      callbacks=[reduce_LR, checkpoint_model1]
  )

  model_1.load_weights(f'/content/drive/MyDrive/assignment-2425/model1_trainable_{seed}.weights.h5')
  y_pred = model_1.predict(x_val)
  f1 = tf.keras.metrics.F1Score(threshold=0.5, average='macro')
  f1.update_state(np.expand_dims(y_val,1).astype('int32'), y_pred)
  print(f1.result())

model_1
Epoch 1/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.6153 - loss: 0.6391 - val_accuracy: 0.7089 - val_loss: 0.6802 - learning_rate: 0.0010
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.7715 - loss: 0.4936 - val_accuracy: 0.7532 - val_loss: 0.6137 - learning_rate: 0.0010
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8398 - loss: 0.3706 - val_accuracy: 0.7785 - val_loss: 0.6186 - learning_rate: 0.0010
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.8967 - loss: 0.2592 - val_accuracy: 0.8101 - val_loss: 0.4574 - learning_rate: 0.0010
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9361 - loss: 0.1666 - val_accuracy: 0.7342 - val_loss: 0.6106 - learning_rate: 0.0010
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9554 - loss: 0.1174 - val_accuracy: 0.8038 - val_loss: 0.6364 - learning_rate: 0.0010
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9740 - loss: 0.0730 -

#### Embedding(training=False) F1-scores = (0.74999994, 0.73684204, 0.76033056)
#### Embedding(trainable=True) F1-score = (0.76562494, 0.76422757, 0.6942148)

In [ ]:
model_1 = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dimension,
        weights=[embedding_matrix],  # Load the pre-trained embeddings
        mask_zero=True,
        trainable=False               # Set to False to keep embeddings fixed
    ),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)), # add another LSTM layer
    Dense(1, activation='sigmoid')  # For binary classification
])

In [ ]:
optim = tf.keras.optimizers.AdamW(learning_rate=0.001)
reduce_LR = tf.keras.callbacks.ReduceLROnPlateau(patience=4)
checkpoint_model1 = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/assignment-2425/model1.weights.h5', save_weights_only=True, save_best_only=True)


model_1.compile(
    loss='binary_crossentropy',
    optimizer=optim,
    metrics=['accuracy']
)

In [ ]:
history_model1 = model_1.fit(
    x_train,
    y_train,
    epochs=30,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[reduce_LR, checkpoint_model1]
)

Epoch 1/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.5997 - loss: 0.6634 - val_accuracy: 0.7025 - val_loss: 0.5419 - learning_rate: 0.0010
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7207 - loss: 0.5521 - val_accuracy: 0.7468 - val_loss: 0.5222 - learning_rate: 0.0010
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7541 - loss: 0.5077 - val_accuracy: 0.7595 - val_loss: 0.4868 - learning_rate: 0.0010
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8000 - loss: 0.4448 - val_accuracy: 0.7658 - val_loss: 0.4822 - learning_rate: 0.0010
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8198 - loss: 0.4096 - val_accuracy: 0.7848 - val_loss: 0.4772 - learning_rate: 0.0010
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8372 - loss: 0.3820 - val_accuracy: 0.8101 - val_loss: 0.4415 - learning_rate: 0.0010
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8563 - loss: 0.3310 - val_acc

In [ ]:
baseline.load_weights('/content/drive/MyDrive/assignment-2425/baseline.weights.h5')
print(baseline.evaluate(x_val, y_val))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7855 - loss: 0.4468 
[0.44200608134269714, 0.7848101258277893]


In [ ]:
model_1.load_weights('/content/drive/MyDrive/assignment-2425/model1.weights.h5')
print(model_1.evaluate(x_val, y_val))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8026 - loss: 0.4434
[0.4414699971675873, 0.8101266026496887]


# [Task 6 - 1.0 points] Transformers

In this section, you will use a transformer model specifically trained for hate speech detection, namely [Twitter-roBERTa-base for Hate Speech Detection](https://huggingface.co/cardiffnlp/twitter-roberta-base-hate).




### Relevant Material
- Tutorial 3

### Instructions
1. **Load the Tokenizer and Model**

2. **Preprocess the Dataset**:
   You will need to preprocess your dataset to prepare it for input into the model. Tokenize your text data using the appropriate tokenizer and ensure it is formatted correctly.

   **Note**: You have to use the plain text of the dataset and not the version that you tokenized before, as you need to tokenize the cleaned text obtained after the initial cleaning process.

3. **Train the Model**:
   Use the `Trainer` to train the model on your training data.

4. **Evaluate the Model on the Test Set** using F1-macro.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from torch.utils.data import Dataset

In [ ]:
train_texts = training['lemmatized'].tolist()
val_texts = validation['lemmatized'].tolist()
test_texts = test['lemmatized'].tolist()

train_labels = training['hard_label_task1'].tolist()
val_labels = validation['hard_label_task1'].tolist()
test_labels = test['hard_label_task1'].tolist()

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-hate"  # Replace with the correct model name

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# Tokenize the training and testing data
train_encodings = tokenizer(train_texts)
val_encodings = tokenizer(val_texts)
test_encodings = tokenizer(test_texts)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_encodings[0]

Encoding(num_tokens=49, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
class SexismDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SexismDataset(train_encodings, train_labels)
val_dataset = SexismDataset(val_encodings, val_labels)
test_dataset = SexismDataset(test_encodings, test_labels)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/assignment-2425/transformer/results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='/content/drive/MyDrive/assignment-2425/transformer/logs',
    load_best_model_at_end=True,
    seed = 42
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'f1': f1_score(labels, predictions, average='macro')
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

<ipython-input-41-27b6ee46aeb3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.322713,0.884221
2,0.420000,0.425000,0.894403
3,0.225000,0.540179,0.887497


TrainOutput(global_step=1077, training_loss=0.31003141270374523, metrics={'train_runtime': 189.7013, 'train_samples_per_second': 45.387, 'train_steps_per_second': 5.677, 'total_flos': 239207004869520.0, 'train_loss': 0.31003141270374523, 'epoch': 3.0})

In [ ]:
eval_result = trainer.evaluate()
print(f"F1-Macro Score: {eval_result['eval_f1']:.4f}")

F1-Macro Score: 0.8842


# [Task 7 - 0.5 points] Error Analysis

### Instructions

After evaluating the model, perform a brief error analysis:

 - Review the results and identify common errors.

 - Summarize your findings regarding the errors and their impact on performance (e.g. but not limited to Out-of-Vocabulary (OOV) words, data imbalance, and performance differences between the custom model and the transformer...)
 - Suggest possible solutions to address the identified errors.



# [Task 8 - 0.5 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.


# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Execution Order

You are **free** to address tasks in any order (if multiple orderings are available).

### Trainable Embeddings

You are **free** to define a trainable or non-trainable Embedding layer to load the GloVe embeddings.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).
However, you are **free** to play with their hyper-parameters.


### Neural Libraries

You are **free** to use any library of your choice to implement the networks (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Keras TimeDistributed Dense layer

If you are using Keras, we recommend wrapping the final Dense layer with `TimeDistributed`.

### Robust Evaluation

Each model is trained with at least 3 random seeds.

Task 4 requires you to compute the average performance over the 3 seeds and its corresponding standard deviation.

### Model Selection for Analysis

To carry out the error analysis you are **free** to either

* Pick examples or perform comparisons with an individual seed run model (e.g., Baseline seed 1337)
* Perform ensembling via, for instance, majority voting to obtain a single model.

### Error Analysis

Some topics for discussion include:
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

### Bonus Points
Bonus points are arbitrarily assigned based on significant contributions such as:
- Outstanding error analysis
- Masterclass code organization
- Suitable extensions
Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

**Possible Extensions/Explorations for Bonus Points:**
- **Try other preprocessing strategies**: e.g., but not limited to, explore techniques tailored specifically for tweets or  methods that are common in social media text.
- **Experiment with other custom architectures or models from HuggingFace**
- **Explore Spanish tweets**: e.g., but not limited to, leverage multilingual models to process Spanish tweets and assess their performance compared to monolingual models.







# The End